In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [4]:
from my_utils.toy_data import invert_seq
train = invert_seq(5000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [5]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=64, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)

In [6]:
from torch_models.models.transformer import *
embed_size = 24
n_head = 4

model = Transformer(size=embed_size, n_head=n_head, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    dropout=0, n_layers=1)
print(model)

Transformer(
  (encoder): TransformerEncoder(
    (embedding): TransformerEmbedding(
      (embedding): Embedding(12, 24, padding_idx=11)
    )
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (attention): MultiHeadedAttention(
          (Q_linear): Linear(in_features=24, out_features=24, bias=True)
          (K_linear): Linear(in_features=24, out_features=24, bias=True)
          (V_linear): Linear(in_features=24, out_features=24, bias=True)
          (out_linear): Linear(in_features=24, out_features=24, bias=True)
          (attention): DotAttn()
          (dropout): Dropout(p=0)
        )
        (fc): PositionwiseFeedForward(
          (w_1): Linear(in_features=24, out_features=96, bias=True)
          (w_2): Linear(in_features=96, out_features=24, bias=True)
          (dropout): Dropout(p=0)
        )
        (layer_norms): ModuleList(
          (0): LayerNorm(torch.Size([24]), eps=1e-05, elementwise_affin

In [7]:
%%time
from my_utils import Trainer, EvaluatorSeq
from torch.optim import Adam, SGD

optimizer = Adam(model.parameters())
evaluator = EvaluatorSeq(model, test_loader, measure='BLEU')

trainer = Trainer(model, train_loader)
trainer.train_epoch(optimizer, max_epoch=5,
              evaluator=evaluator, score_monitor=None, show_log=True)

epoch 0  	loss: 1.9343135009838055	BLEU: 0.1281729147693661	
epoch 1  	loss: 1.0099508603916894	BLEU: 0.3929462679053081	
epoch 2  	loss: 0.42142942396900324	BLEU: 0.7154152007168064	
epoch 3  	loss: 0.1709221069571338	BLEU: 0.9033826189370201	
epoch 4  	loss: 0.057922882251913034	BLEU: 0.9226460308229564	
CPU times: user 40.5 s, sys: 420 ms, total: 40.9 s
Wall time: 14.8 s


In [8]:
assert 0.9 < evaluator.evaluate()

In [9]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['8', '7', '9', '4']
['4', '9', '4', '3', '5']
['4', '6', '0']
['7', '4', '7', '8']
['6', '2', '5', '1']
['0', '9', '3']
['0', '7', '8', '5']
['8', '4', '5', '1', '3']
['6', '5', '9']
['0', '9', '5', '0', '9']
======= output ======
['4', '9', '7', '8']
['5', '3', '4', '9', '4']
['0', '6', '4']
['8', '7', '4', '7']
['1', '5', '2', '6']
['3', '9', '0']
['5', '8', '7', '0']
['3', '1', '5', '4', '8']
['9', '5', '6']
['9', '0', '5', '9', '0']
